# Importação das bibliotecas

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import folium
from folium.plugins import MarkerCluster

# Lendo arquivo

In [2]:
df = pd.read_excel('Dados_PRF_2023.xls')

# Dicionário de Dados

In [3]:
df.columns

Index(['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional',
       'delegacia', 'uop'],
      dtype='object')

* nb_id
* dt_data_inversa
* tx_dia_semana
* nb_horario
* tx_uf
* br
* km
* municipio
* causa_acidente
* tipo_acidente
* classificacao_acidente
* fase_dia
* sentido_via
* condicao_metereologica
* tipo_pista
* tracado_via
* uso_solo
* pessoas
* mortos
* feridos_leves
* feridos_graves
* ilesos
* ignorados
* feridos
* veiculos
* latitude
* longitude
* regional
* delegacia
* uop

# Tratamento de Dados

In [4]:
# Criando uma nova coluna 'mes' para armazenar o mês correspondente
df['mes'] = df['data_inversa'].dt.month

# Tratando coluna de horário
df.rename(columns={'hora': 'horario'}, inplace=True)
df['horario'].fillna('00:00:00', inplace=True)
df['horario'] = df['horario'].astype(str)
df['hora'] = df['horario'].str[:2]

# Preenchendo nulos de delegacia
def extract_from_column(value):
    if pd.isnull(value):
        return None
    else:
        return str(value)[6:]  # Converter para string e pegar a substring a partir do 7º caractere

# Preencher os nulos em coluna1 com base em coluna2
df['delegacia'] = df.apply(lambda row: extract_from_column(row['uop']) if pd.isnull(row['delegacia']) else row['delegacia'], axis=1)

# Preenchendo nulos de br e km
df[['br', 'km']] = df[['br', 'km']].fillna(0)

# Preenchendo nulos de regional
df['regional'] = df['regional'].fillna('Indefinido')

#Preenchendo nulos de delegacia e uop
df[['delegacia', 'uop']] = df[['delegacia', 'uop']].fillna('Indefinido')

df[['br', 'km']]  = df[['br', 'km']] .astype(str)

# Análise Exploratória

In [5]:
display(df)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,mes,hora
0,496519,2023-01-01,domingo,02:00:00,ES,101.0,114.0,SOORETAMA,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Simples,Reta,Não,1,0,1,0,0,0,1,1,-19.094849,-40.050958,SPRF-ES,DEL04-ES,UOP01-DEL04-ES,1,02
1,496543,2023-01-01,domingo,03:40:00,SP,116.0,113.1,TAUBATE,Entrada inopinada do pedestre,Atropelamento de Pedestre,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,Sim,5,1,0,0,0,4,0,1,-23.044566,-45.582598,SPRF-SP,DEL02-SP,UOP02-DEL02-SP,1,03
2,496590,2023-01-01,domingo,01:40:00,MT,163.0,1112.0,GUARANTA DO NORTE,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Plena Noite,Crescente,Ignorado,Simples,Não Informado,Não,1,0,0,1,0,0,1,1,-9.700206,-54.875888,SPRF-MT,DEL06-MT,UOP03-DEL06-MT,1,01
3,496610,2023-01-01,domingo,10:40:00,PR,376.0,314.8,ORTIGUEIRA,Velocidade Incompatível,Tombamento,Sem Vítimas,Pleno dia,Crescente,Sol,Dupla,Curva,Não,1,0,0,0,1,0,0,1,-23.985512,-51.083555,SPRF-PR,DEL07-PR,UOP02-DEL07-PR,1,10
4,496659,2023-01-01,domingo,14:55:00,MG,116.0,569.4,MANHUACU,Acumulo de água sobre o pavimento,Colisão frontal,Com Vítimas Feridas,Pleno dia,Decrescente,Chuva,Simples,Não Informado,Não,3,0,0,2,1,0,2,2,-20.100075,-42.178841,SPRF-MG,DEL06-MG,UOP03-DEL06-MG,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32435,536157,2023-06-25,domingo,05:10:00,SP,101.0,41.0,UBATUBA,Ingestão de álcool pelo condutor,Tombamento,Sem Vítimas,Amanhecer,Decrescente,Céu Claro,Simples,Curva,Não,1,0,0,0,1,0,0,1,-23.419714,-45.042866,SPRF-SP,DEL06-SP,UOP01-DEL06-SP,6,05
32436,536172,2023-06-23,sexta-feira,02:30:00,RS,386.0,279.4,SAO JOSE DO HERVAL,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Decrescente,Chuva,Simples,Reta,Não,1,0,0,1,0,0,1,1,-29.023377,-52.315435,SPRF-RS,DEL04-RS,UOP03-DEL04-RS,6,02
32437,536173,2023-06-24,sábado,13:40:00,PR,163.0,133.3,CAPITAO LEONIDAS MARQUES,Retorno proibido,Colisão transversal,Com Vítimas Feridas,Pleno dia,Decrescente,Sol,Dupla,Não Informado,Não,5,0,3,0,2,0,3,2,-25.554524,-53.587561,SPRF-PR,DEL05-PR,UOP05-DEL05-PR,6,13
32438,536265,2023-01-09,segunda-feira,10:00:00,MG,267.0,80.5,JUIZ DE FORA,Condutor Dormindo,Saída de leito carroçável,Sem Vítimas,Pleno dia,Crescente,Chuva,Simples,Curva,Não,1,0,0,0,1,0,0,1,-21.754786,-43.212418,SPRF-MG,DEL05-MG,UOP01-DEL05-MG,1,10


## Quantidade de linhas e colunas

In [6]:
df.shape

(32440, 32)

## Tipos das colunas

In [7]:
df.dtypes

id                                 int64
data_inversa              datetime64[ns]
dia_semana                        object
horario                           object
uf                                object
br                                object
km                                object
municipio                         object
causa_acidente                    object
tipo_acidente                     object
classificacao_acidente            object
fase_dia                          object
sentido_via                       object
condicao_metereologica            object
tipo_pista                        object
tracado_via                       object
uso_solo                          object
pessoas                            int64
mortos                             int64
feridos_leves                      int64
feridos_graves                     int64
ilesos                             int64
ignorados                          int64
feridos                            int64
veiculos        

## Nulos por variável

In [8]:
df.isnull().sum()

id                        0
data_inversa              0
dia_semana                0
horario                   0
uf                        0
br                        0
km                        0
municipio                 0
causa_acidente            0
tipo_acidente             0
classificacao_acidente    0
fase_dia                  0
sentido_via               0
condicao_metereologica    0
tipo_pista                0
tracado_via               0
uso_solo                  0
pessoas                   0
mortos                    0
feridos_leves             0
feridos_graves            0
ilesos                    0
ignorados                 0
feridos                   0
veiculos                  0
latitude                  0
longitude                 0
regional                  0
delegacia                 0
uop                       0
mes                       0
hora                      0
dtype: int64

## Estatísticas Gerais

In [9]:
df.describe()

,id,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,mes
count,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000,32440.000000
mean,515154.342170,2.457614,0.081936,0.874723,0.277435,1.067109,0.156412,1.152158,1.664365,-19.069813,-46.563997,3.571979
std,10528.741985,2.166629,0.325561,1.083875,0.617148,1.758439,0.443078,1.194990,0.720026,7.702761,6.153457,1.703507
min,496506.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-33.588291,-69.503043,1.000000
25%,506122.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-25.363744,-50.439377,2.000000
50%,515290.500000,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-20.620693,-47.545808,4.000000
75%,524296.250000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-12.912337,-42.570753,5.000000
max,536330.000000,65.000000,7.000000,61.000000,29.000000,58.000000,14.000000,62.000000,12.000000,4.383141,-34.828400,6.000000


# Análise Temporal

In [10]:
# Ocorrências agrupadas por mês
df['mes'].value_counts()

6    5667
5    5602
4    5594
3    5583
1    5250
2    4744
Name: mes, dtype: int64

In [11]:
# Ocorrências agrupadas por dia da semana
df['dia_semana'].value_counts()

domingo          5375
sábado           5140
sexta-feira      4975
segunda-feira    4530
quinta-feira     4370
quarta-feira     4094
terça-feira      3956
Name: dia_semana, dtype: int64

In [12]:
# Ocorrências agrupadas por horário
df['hora'].value_counts()

18    2499
17    2165
19    2067
07    2062
16    1771
15    1645
08    1644
14    1555
06    1486
20    1484
12    1392
11    1376
10    1366
13    1365
09    1301
21    1242
22    1062
05     982
23     900
04     690
00     685
03     589
01     585
02     527
Name: hora, dtype: int64

## Análise Geográfica

In [13]:
# Ocorrências agrupadas por estados
df_uf = df['uf'].value_counts()
df_uf.head(10)

MG    4234
SC    3839
PR    3442
RJ    2602
RS    2536
SP    2283
BA    1739
GO    1482
PE    1441
ES    1090
Name: uf, dtype: int64

In [14]:
# Ocorrências agrupadas por municípios
df_municipios = df['municipio'].value_counts()
df_municipios.head(10)

BRASILIA                464
GUARULHOS               395
SAO JOSE                326
DUQUE DE CAXIAS         323
PALHOCA                 307
RECIFE                  284
CURITIBA                281
BETIM                   277
SAO JOSE DOS PINHAIS    235
SERRA                   231
Name: municipio, dtype: int64

## Insights de Negócios

In [15]:
display(df)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,mes,hora
0,496519,2023-01-01,domingo,02:00:00,ES,101.0,114.0,SOORETAMA,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Simples,Reta,Não,1,0,1,0,0,0,1,1,-19.094849,-40.050958,SPRF-ES,DEL04-ES,UOP01-DEL04-ES,1,02
1,496543,2023-01-01,domingo,03:40:00,SP,116.0,113.1,TAUBATE,Entrada inopinada do pedestre,Atropelamento de Pedestre,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,Sim,5,1,0,0,0,4,0,1,-23.044566,-45.582598,SPRF-SP,DEL02-SP,UOP02-DEL02-SP,1,03
2,496590,2023-01-01,domingo,01:40:00,MT,163.0,1112.0,GUARANTA DO NORTE,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Plena Noite,Crescente,Ignorado,Simples,Não Informado,Não,1,0,0,1,0,0,1,1,-9.700206,-54.875888,SPRF-MT,DEL06-MT,UOP03-DEL06-MT,1,01
3,496610,2023-01-01,domingo,10:40:00,PR,376.0,314.8,ORTIGUEIRA,Velocidade Incompatível,Tombamento,Sem Vítimas,Pleno dia,Crescente,Sol,Dupla,Curva,Não,1,0,0,0,1,0,0,1,-23.985512,-51.083555,SPRF-PR,DEL07-PR,UOP02-DEL07-PR,1,10
4,496659,2023-01-01,domingo,14:55:00,MG,116.0,569.4,MANHUACU,Acumulo de água sobre o pavimento,Colisão frontal,Com Vítimas Feridas,Pleno dia,Decrescente,Chuva,Simples,Não Informado,Não,3,0,0,2,1,0,2,2,-20.100075,-42.178841,SPRF-MG,DEL06-MG,UOP03-DEL06-MG,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32435,536157,2023-06-25,domingo,05:10:00,SP,101.0,41.0,UBATUBA,Ingestão de álcool pelo condutor,Tombamento,Sem Vítimas,Amanhecer,Decrescente,Céu Claro,Simples,Curva,Não,1,0,0,0,1,0,0,1,-23.419714,-45.042866,SPRF-SP,DEL06-SP,UOP01-DEL06-SP,6,05
32436,536172,2023-06-23,sexta-feira,02:30:00,RS,386.0,279.4,SAO JOSE DO HERVAL,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Decrescente,Chuva,Simples,Reta,Não,1,0,0,1,0,0,1,1,-29.023377,-52.315435,SPRF-RS,DEL04-RS,UOP03-DEL04-RS,6,02
32437,536173,2023-06-24,sábado,13:40:00,PR,163.0,133.3,CAPITAO LEONIDAS MARQUES,Retorno proibido,Colisão transversal,Com Vítimas Feridas,Pleno dia,Decrescente,Sol,Dupla,Não Informado,Não,5,0,3,0,2,0,3,2,-25.554524,-53.587561,SPRF-PR,DEL05-PR,UOP05-DEL05-PR,6,13
32438,536265,2023-01-09,segunda-feira,10:00:00,MG,267.0,80.5,JUIZ DE FORA,Condutor Dormindo,Saída de leito carroçável,Sem Vítimas,Pleno dia,Crescente,Chuva,Simples,Curva,Não,1,0,0,0,1,0,0,1,-21.754786,-43.212418,SPRF-MG,DEL05-MG,UOP01-DEL05-MG,1,10


In [16]:
# Km das rodovias que mais possuem acidentes
grouped = df.groupby(['br', 'km'])['causa_acidente'].count().reset_index()
grouped_sorted = grouped.sort_values(by='causa_acidente', ascending=False)

display(grouped_sorted.head(10))

,br,km,causa_acidente
0,0.0,0.0,78
3004,116.0,230.0,38
2995,116.0,229.0,33
666,101.0,206.0,32
1887,101.0,70.0,31
2948,116.0,219.0,28
2985,116.0,227.0,27
2811,116.0,176.0,26
2967,116.0,223.0,26
1188,101.0,322.0,25


In [17]:
# Ocorrências agrupadas por causa dos acidentes
df_municipios = df['causa_acidente'].value_counts()
df_municipios.head(10)

Reação tardia ou ineficiente do condutor                     4672
Ausência de reação do condutor                               4354
Acessar a via sem observar a presença dos outros veículos    3017
Velocidade Incompatível                                      2124
Condutor deixou de manter distância do veículo da frente     2082
Manobra de mudança de faixa                                  1936
Ingestão de álcool pelo condutor                             1755
Demais falhas mecânicas ou elétricas                         1323
Transitar na contramão                                       1121
Condutor Dormindo                                             993
Name: causa_acidente, dtype: int64

In [18]:
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=13)

# Adicionar um cluster de marcadores para melhorar a visualização de múltiplos pontos
marker_cluster = MarkerCluster().add_to(m)

# Adicionar marcadores para cada ponto de acidente
for idx, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']]).add_to(marker_cluster)

# Exibir o mapa
m.save('mapa_acidentes.html')